In [2]:
from neo4j import GraphDatabase
import pandas as pd
import os

In [4]:
catalog = "scraped_data"

if not os.path.exists(catalog):
    os.makedirs(catalog)

%run "gamesradar_ranking_scraper.py"
%run "ign_ranking_scraper.py"
%run "imdb_genres_scraper.py"
%run "imdb_ranking_scraper.py"
%run "metacritic_genres_scraper.py"
%run "metacritic_platforms_ranking_scraper.py"
%run "steam_250_tag_scraper.py"

In [ ]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "987654321")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
print("The connection has been established!")

In [ ]:
driver.execute_query("MATCH (N) DETACH DELETE N")
print("Database truncated!")

In [ ]:
genres = pd.read_csv("scraped_data/genres.csv")

def add_to_database(genre):
    driver.execute_query(f"CREATE (:Genre {{name: '{genre}'}})")
    
genres["genre"].apply(add_to_database)

print("Genres added successfully!")

In [ ]:
ign_ranking = pd.read_csv("scraped_data/ign_ranking.csv")

driver.execute_query(f"CREATE (:Ranking {{publisher:'IGN', url:'https://www.ign.com/articles/the-best-100-video-games-of-all-time', positions:{len(ign_ranking)}}});")

def add_to_database(row):
    driver.execute_query(f"""
                         MATCH (r:Ranking {{publisher:'IGN'}})
                         MERGE (g:Game {{title: '{row['name']}'}})
                         CREATE (r)-[:RANKS {{position: {row['position']}}}]->(g)-[:HAS_POSITION {{position: {row['position']}}}]->(r)
                         """)
    
ign_ranking.apply(add_to_database, axis=1)
print("Ranking IGN and its nodes added!")

In [ ]:
gr = pd.read_csv("scraped_data/gamesradar_ranking.csv")

driver.execute_query(f"CREATE (:Ranking {{publisher:'GamesRadar+', url:'https://www.gamesradar.com/best-games-2023', positions:{len(gr)}}});")

def add_to_database(row):
    driver.execute_query(f"""
                         MATCH (r:Ranking {{publisher:'GamesRadar+'}})
                         MERGE (g:Game {{title: '{row['name']}'}})
                         SET g.premiere=2023
                         CREATE (r)-[:RANKS {{position: {row['position']}}}]->(g)-[:HAS_POSITION {{position: {row['position']}}}]->(r);
                         """)
    
gr.apply(add_to_database, axis=1)
print("Ranking GamesRadar+ and its nodes added!")

In [ ]:
imdb = pd.read_csv("scraped_data/imdb_ranking.csv")

driver.execute_query(f"CREATE (:Ranking {{publisher:'IMDb', url:'https://www.imdb.com/list/ls097840768', positions:{len(imdb)}}});")

def add_to_database(row):
    
    driver.execute_query(f"""
                         MATCH (r:Ranking {{publisher:'IMDb'}})
                         MERGE (g:Game {{title: '{row['name']}'}})
                         SET g.premiere={row['year']}
                         MERGE (r)-[:RANKS {{position: {row['position']}}}]->(g)-[:HAS_POSITION {{position: {row['position']}}}]->(r);
                         """)

    for g in [row["g1"], row["g2"], row["g3"]]:
        if g != "none":
            driver.execute_query(f"""
                                  MATCH (e:Genre {{name: '{g}'}})
                                  MATCH (g:Game {{title: '{row['name']}'}})
                                  MERGE (g)-[:HAS_GENRE]->(e)-[:GENRE_OF]->(g);
                                  """)
    
imdb.apply(add_to_database, axis=1)
print("Ranking IMDb and its nodes added!")

In [ ]:
platforms = ["ps1", "ps2", "ps3", "ps4", "ps5", "pc", "xbox-series-x"]
names = ["PlayStation 1", "PlayStation 2", "PlayStation 3", "PlayStation 4", "PlayStation 5", "PC", "Xbox"]
nodes = ["PS1", "PS2", "PS3", "PS4", "PS5", "PC", "Xbox"]

for jj in range(len(names)):
    metacritic_data = pd.read_csv(f"scraped_data/metacritic_{platforms[jj]}_ranking.csv")

    driver.execute_query(f"CREATE (:Ranking {{publisher: 'Metacritic {nodes[jj]}', url: 'https://www.metacritic.com/browse/game/{platforms[jj]}'}});")
    driver.execute_query(f"CREATE (:Platform {{name: '{names[jj]}'}});")

    def add_to_database(row):
        driver.execute_query(f"""
                            MATCH (r:Ranking {{publisher: 'Metacritic {nodes[jj]}'}})	
                            MATCH (p:Platform {{name: '{names[jj]}'}})	
                            MERGE (g:Game {{title: '{row['name']}'}})
                            SET g.premiere={row['year']}
                            SET g.description='{row['description']}'
                            SET g.metacritic_score={row['metacritic_score']}
                            CREATE (g)-[:HAS_POSITION {{position:{row['position']}}}]->(r)-[:RANKS {{position:{row['position']}}}]->(g),
                                   (g)-[:AVAILABLE_ON]->(p)-[:SUPPORTS]->(g);
                            """)

    metacritic_data.apply(add_to_database, axis=1)
    print(f"Ranking Metacritic {nodes[jj]} and its nodes added!")

In [ ]:
# Steam 250 Tags
es = ["comedy", "mystery", "sci-fi", "family_friendly", "horror", "fantasy", "romance", "drama", "war", "action", "historical", "adventure", "sports", "cartoony", "cartoon", "detective", "clicker", "survival", "zombies", "crafting", "education", "survival_horror", "magic", "dark"]
names = []

for e in es:
    if e == "family_friendly": names.append("Family")
    elif e == "historical": names.append("History")
    elif e == "sports": names.append("Sport")
    elif e == "cartoony" or e == "cartoon": names.append("Animation")
    elif e == "detective": names.append("Crime")
    elif e == "survival_horror": names.append("Horror")
    elif e == "magic": names.append("Fantasy")
    elif e == "dark": names.append("Thriller")
    else: names.append(e.title())
    
for i in range(len(es)):
    data = pd.read_csv(f"scraped_data/steam_250_{es[i]}.csv")

    driver.execute_query(f"MERGE (:Genre {{name: '{names[i]}'}})")

    driver.execute_query(f"CREATE (:Ranking {{publisher:'Steam 250 Tag {es[i].replace("_", " ").title()}', url:'https://steam250.com/tag/{es[i]}', positions: {len(data)}}});")

    def add_to_database(row):
        if row["year"] == -1:
            driver.execute_query(f"""
                                 MATCH (r:Ranking {{publisher:'Steam 250 Tag {es[i].replace("_", " ").title()}'}})
                                  MATCH (e:Genre {{name: '{names[i]}'}})
                                  MERGE (g:Game {{title: '{row['name']}'}})
                                  SET g.available_on_steam=true
                                  SET g.premiere={row['year']}
                                  MERGE (r)-[:RANKS {{position: {row['position']}}}]->(g)-[:HAS_POSITION {{position: {row['position']}}}]->(r)
                                  MERGE (g)-[:HAS_GENRE]->(e)-[:GENRE_OF]->(g);
                                  """)
        else:
            driver.execute_query(f"""
                                 MATCH (r:Ranking {{publisher:'Steam 250 Tag {es[i].replace("_", " ").title()}'}})
                                 MATCH (e:Genre {{name: '{names[i]}'}})
                                 MERGE (g:Game {{title: '{row['name']}'}})
                                 SET g.available_on_steam=true
                                 MERGE (r)-[:RANKS {{position: {row['position']}}}]->(g)-[:HAS_POSITION {{position: {row['position']}}}]->(r)
                                 MERGE (g)-[:HAS_GENRE]->(e)-[:GENRE_OF]->(g);
                                  """)
    
    data.apply(add_to_database, axis=1)

print("Tags of ranking Steam 250 and its nodes added!")


In [ ]:
# Set some available_on_steam = false
to_set = ["Warcraft", "Fortnite", "The Old Republic", "Starcraft", "Heroes of the Storm", "Heroes of Might and Magic IV"]
unwind = f"['{to_set[0]}'"
for g in to_set[1:]:
    unwind += f", '{g}'"
unwind += "]"

driver.execute_query(f"""
                      UNWIND {unwind} AS titles
                      MATCH (g:Game)
                      WHERE g.title CONTAINS titles
                      SET g.available_on_steam=false;
                      """)

print("available_on_steam=false doned!")

driver.execute_query(f"""
                      MATCH (g:Game)
                      WHERE g.premiere = -1
                      REMOVE g.premiere;
                      """)

print("Wrong dates removed!")

In [ ]:
# Metacritic (only genres)
genres = ["adventure", "action", "action-rpg", "sports", "survival"]
names = []

for n in genres:
    if n == "action-rpg": names.append("Action")
    elif n == "sports": names.append("Sport")
    else: names.append(n.title())

for i in range(len(genres)):
    data = pd.read_csv(f"scraped_data/metacritic_{genres[i]}.csv")

    def add_to_database(row):
        driver.execute_query(f"""
                              MATCH (e:Genre {{name: '{names[i]}'}})
                              MERGE (g:Game {{title: '{row['name']}'}})
                              SET g.premiere={row['year']}
                              SET g.metacritic_score={row['metacritic_score']}
                              SET g.description='{row['description']}'
                              MERGE (g)-[:HAS_GENRE]->(e)-[:GENRE_OF]->(g);
                              """)
    
    data.apply(add_to_database, axis=1)

print("Metacritic games to genres added.")